# Tracing LlamaIndex with OTEL Spans using _TruLens_

This notebook demonstrates the "otel-tracing" experimental feature in _TruLens_.
This enables the collection of _OpenTelemetry_ spans during app evaluation. Data
that is collected by _TruLens_ is recorded as spans. Spans created by other tools
can also be made available alongside those created by TruLens. Spans can also be
exported to an OTEL exporter.

- Spans demonstrated in this notebook are:

  - OTEL `sqlalchemy` module instrumentation. Note that `sqlalchemy` is used
    internally by _TruLens_ for storage.

  - OTEL `requests` module instrumentation. `requests` is used by TruLens to
    make requests in the _HuggingFace_ provider.

  - _Traceloop_ LlamaIndex instrumentation.

  - _Traceloop_ OpenAI instrumentation.

  - Arize _OpenInference_ LlamaIndex instrumentation.

- OTEL exporters demonstrated in this notebook are:

  - Console exporter (prints exported spans in the console or stream). We
    configure this exporter to write spans to a variable `stream`.

  - _Zipkin_ exporter. Setup below includes `docker` commands to download and
    start a _Zipkin_ collector for demonstration purposes. To open the UI for
    this exporter, open _Docker Desktop_, click on the triple dots under
    "Actions" for the zipkin container and select "Open with browser".
  

In [ ]:
# python deps, OTEL:
# ! pip install opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp

# OTEL contrib instrumentors
#  ! pip install opentelemetry-instrumentation-sqlalchemy opentelemetry-instrumentation-requests

# Traceloop instrumentors
# ! pip install opentelemetry-instrumentation-llamaindex opentelemetry-instrumentation-openai

# Arize openinference instrumentors
# ! pip install "openinference-instrumentation-llama-index>=2"

# OTEL zipkin exporter
# ! pip install opentelemetry-exporter-zipkin-proto-http

# Start the zipkin docker container:
# ! docker run --rm -d -p 9411:9411 --name zipkin openzipkin/zipkin

# Stop the zipkin docker container:
# ! docker stop $(docker ps -a -q --filter ancestor=openzipkin/zipkin)

In [ ]:
from io import StringIO  # noqa: E402
import json
import os
import re
import urllib.request

import dotenv
from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI

# arize openinference instrumentors
from openinference.instrumentation.llama_index import (
    LlamaIndexInstrumentor as oi_LlamaIndexInstrumentor,
)
from opentelemetry import trace

# zipkip exporter
# traceloop instrumentors
from opentelemetry.instrumentation.llamaindex import LlamaIndexInstrumentor
from opentelemetry.instrumentation.openai import OpenAIInstrumentor

# otel contrib instrumentors:
from opentelemetry.instrumentation.requests import RequestsInstrumentor
from opentelemetry.instrumentation.sqlalchemy import SQLAlchemyInstrumentor

# console exporter
from opentelemetry.sdk.trace.export import ConsoleSpanExporter
from trulens.apps.llamaindex import TruLlama
from trulens.core import Feedback
from trulens.core import Select
from trulens.core.session import TruSession
from trulens.experimental.otel_tracing.core.trace import TracerProvider
from trulens.providers.huggingface import Huggingface

# This is needed due to zipkin issues related to protobuf.
os.environ["OTEL_PYTHON_LOGGING_AUTO_INSTRUMENTATION_ENABLED"] = "true"

dotenv.load_dotenv()

In [ ]:
# Sets the global default tracer provider to be the trulens one.
trace.set_tracer_provider(TracerProvider())

# Creates a tracer for custom spans below.
tracer = trace.get_tracer(__name__)

In [ ]:
# Download some base data for query engine.

url = "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt"
file_path = "data/paul_graham_essay.txt"

if not os.path.exists("data"):
    os.makedirs("data")

if not os.path.exists(file_path):
    urllib.request.urlretrieve(url, file_path)

In [ ]:
# Setup console/file/string exporter
stream = StringIO()
# Will print A LOT to stdout unless we set a different stream.
exporter = ConsoleSpanExporter(out=stream)

# Setup zipkin exporter
# exporter = ZipkinExporter(endpoint="http://localhost:9411/api/v2/spans")

# Setup session with exporter.
tru = TruSession(_experimental_otel_exporter=exporter)

# If not using the exporter, manually enable the otel experimental feature:
TruSession().experimental_enable_feature("otel_tracing")

In [ ]:
# enable otel contrib instrumentation
SQLAlchemyInstrumentor().instrument()
RequestsInstrumentor().instrument()

# enable traceloop instrumentation
LlamaIndexInstrumentor().instrument()
OpenAIInstrumentor().instrument()

# enable arize open inference instrumentation
oi_LlamaIndexInstrumentor().instrument()

In [ ]:
# Create query engine

Settings.llm = OpenAI()

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine(similarity_top_k=3)

In [ ]:
# Create a feedback function and wrap app with trulens recorder.

provider = Huggingface()

f_lang_match = (
    Feedback(provider.language_match)
    .on(
        Select.RecordSpans.trulens.call.query.attributes.trulens.bindings.str_or_query_bundle
    )
    .on(Select.RecordSpans.trulens.call.query.attributes.trulens.ret.response)
)

"""
The parts of the selector are:

- Select.RecordSpans - the spans organized by span name.

- trulens.call.query - the span name we are interested in. TruLens names all
    call spans with the name "trulens.call.<methodname>".

- attributes - the attributes of the span.

- trulens.bindings.str_or_query_bundle - the attribute we are interested in.
    TruLens puts the call arguments in the attribute called
    "trulens.bindings".
"""


tru_query_engine_recorder = TruLlama(
    query_engine,
    app_name="LlamaIndex_App",
    app_version="base",
    feedbacks=[f_lang_match],
)

In [ ]:
# Normal trulens recording usage

with tru_query_engine_recorder as recording:
    # Custom spans can be included:
    with tracer.start_as_current_span("Querying LlamaIndex") as span:
        # With custom attributes.
        span.set_attribute("custom_attribute", "This can by anything.")

        # Query the engine as normal.
        query_engine.query("What did the author do growing up?")

In [ ]:
# Get the record from the recording.

rec = recording.get()

In [ ]:
# Check the feedback result.

rec.feedback_results[0].result()

In [ ]:
# Show all spans in the record. Here we are using a selector to retrieve the
# spans from within the record.

rec.get(Select.RecordSpans)

# Alternatively, spans can be accessed directly in the record as a list. The
# above indexes them by name instead.

# rec.experimental_otel_spans

In [ ]:
# Check the attributes we used to define the feedback functions.

print(
    rec.get(
        Select.RecordSpans.trulens.call.query.attributes.trulens.bindings.str_or_query_bundle
    )
)

print(
    rec.get(
        Select.RecordSpans.trulens.call.query.attributes.trulens.ret.response
    )
)

In [ ]:
# All of the spans listed above should be visible in the chosen exporter.
#
# The ConsoleSpanExporter writes json dumps of each span. Lets read those back
# here to inspect them:

match_root_json = re.compile(r"(?:(^|\n))\{.+?\n\}", re.DOTALL)

if "stream" in locals():
    dumps = match_root_json.finditer(stream.getvalue())

    for dump in dumps:
        span = json.loads(dump.group())
        print(span["name"])

        """
    This should include:

        - 0: a special span made by TruLens that indicates a recording context.

        - 1: the custom span entitled "Querying LlamaIndex" made above.

        - 2: the span made by TruLens that corresponds to the call to
        `query_engine.query`.

        - 3: one of the spans produced by the other instrumentors that
        represents that same call.
    """

In [ ]:
# Check a spans produced by TruLens. Note that span instances created by TruLens
# are represented as:
#
#  <class name>(<name>, <trace_id>/<span_id> -> <parent trace_id>/<parent span_id>)
#
# where trace_id and span_id are only the last 2 bytes of each for easier readability.

rec.get(Select.RecordSpans.trulens)

In [ ]:
# Check details of one the main span (representing the call to `query`).

rec.get(Select.RecordSpans.trulens.call.query.attributes)

In [ ]:
# Check attributes of the same information as instrumented by OpenInference:

rec.get(Select.RecordSpans.RetrieverQueryEngine._query.attributes)

In [ ]:
# Check attributes of the same information as instrumented by TraceLoop:

rec.get(Select.RecordSpans.RetrieverQueryEngine.workflow.attributes)

In [ ]:
# Check for spans that were produced outside of the recording. Here we print all
# of the root spans (those that do not have a parent). This should include the
# special TruLens span that corresponds to a recording but also other spans
# produced before and after the recording.

for span in tracer.spans.values():
    if span.parent is None:
        print(span, span.status, span.attributes.keys())

In [ ]:
# Check some of the specific spans.

# SQLAlchmey spans:

for span in tracer.spans.values():
    if span.name == "connect":
        print(span, span.status, span.attributes)

In [ ]:
# requests spans:

for span in tracer.spans.values():
    if span.name in ["POST", "GET"]:
        print(span, span.status, span.attributes)